In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:

import wave

def capture_audio(file_path, duration=5):
    chunk = 1024
    format = pyaudio.paInt16
    channels = 1
    rate = 44100

    p = pyaudio.PyAudio()

    stream = p.open(format=format, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)

    print("Recording...")
    frames = []

    for _ in range(0, int(rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    with wave.open(file_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))
        import speech_recognition as sr

        def transcribe_audio(file_path):
            recognizer = sr.Recognizer()
            with sr.AudioFile(file_path) as source:
                audio = recognizer.record(source)  # read the entire audio file
            try:
                text = recognizer.recognize_google(audio)
                return text
            except sr.UnknownValueError:
                return "Could not understand audio"
            except sr.RequestError as e:
                return f"Could not request results from Google Speech Recognition service; {e}"
            import language_tool_python

            def check_grammar(text):
                tool = language_tool_python.LanguageTool('en-US')
                matches = tool.check(text)
                return matches

            def main():
                audio_file_path = "audio.wav"

                capture_audio(audio_file_path, duration=5)  # Record for 5 seconds

                transcript = transcribe_audio(audio_file_path)
                print("Transcript:", transcript)

                grammar_issues = check_grammar(transcript)

                if grammar_issues:
                    print("Grammar Issues Found:")
                    for issue in grammar_issues:
                        print(f"- {issue.ruleId}: {issue.message} (Suggestion: {issue.replacements})")
                else:
                    print("No grammar issues found.")

            if __name__ == "__main__":
                main()